<a href="https://colab.research.google.com/github/Lishusianochka/tyrbiny-dz-Frizen-VA/blob/5-%D0%B4%D0%B7/%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Фризен В.А. ТФэ-01-20 Lishusianochka

In [1]:
!pip install iapws
import iapws
from iapws import IAPWS97 as gas

MPa = 10 ** 6
kPa = 10 ** 3
unit = 1 / MPa
to_kelvin = lambda x: x + 273.15 if x else None
#Исходные данные
electrical_power = 250 * (10 ** 6)
p0 = 12.8 * MPa
t0 = 540
pk = 6.9 * kPa
t_feed_water = 263
p_feed_water = 1.4 * p0

z = 8

internal_efficiency = 0.83
mechanical_efficiency = 0.992
generator_efficiency = 0.99
delta_p0 = 0.05 * p0

real_p0 = p0 - delta_p0
#teor - теоретические точки
#d - действительные(реальные)
point_0_teor = gas(P = p0 * unit, T = to_kelvin(t0))
point_0_d = gas(P = real_p0 * unit, h = point_0_teor.h)

point_2_teor = gas(P = pk * unit, s = point_0_teor.s)
heat_drop_d = (point_0_teor.h - point_2_teor.h) * internal_efficiency
h_2_d = point_0_d.h - heat_drop_d
point_2_d = gas(P = pk * unit, h = h_2_d)

point_k_water = gas(P = pk * unit, x = 0)
point_feed_water = gas(P = p_feed_water * unit, T = to_kelvin(t_feed_water))

numenator_without = (point_0_teor.s - point_k_water.s) * point_2_d.T
denumenator_without = (point_0_d.h - point_k_water.h)
without_part = 1 - (numenator_without / denumenator_without)

numenator_infinity = point_2_d.T * (point_0_teor.s - point_feed_water.s)
denumenator_infinity = (point_0_d.h - point_feed_water.h)
infinity_part = 1 - (numenator_infinity / denumenator_infinity)

ksi_infinity = 1 - (without_part / infinity_part)

coeff = (point_feed_water.T - point_2_d.T) / (to_kelvin(374.2) - point_2_d.T)
print("Коэффициент:", coeff)

ksi = 0.83 * ksi_infinity

eff_num = heat_drop_d
eff_denum = (point_0_d.h - point_k_water.h)

efficiency = (eff_num / eff_denum) * (1 / (1 - ksi))
print("КПД:", round(efficiency * 100,3),"%")

estimated_heat_drop = efficiency * (point_0_d.h - point_feed_water.h)

inlet_mass_flow = electrical_power / (estimated_heat_drop * 1000 * mechanical_efficiency * generator_efficiency)

condenser_mass_flow = (
electrical_power /
((point_0_d.h - point_k_water.h) * 1000 * mechanical_efficiency * generator_efficiency) * ((1 / efficiency) - 1)
)

print("Массовый расход в турбину на входе", inlet_mass_flow)
print("Массовый расход в конденсатор:", condenser_mass_flow)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 24.9 MB/s 
  Created wheel for iapws: filename=iapws-1.5.3-py3-none-any.whl size=114282 sha256=6a811a46c625f4159dc8eecb2c27256d57fcf71e82e69ed006aa63c9cfd74c90
  Stored in directory: /root/.cache/pip/wheels/66/b4/cb/ef55293a897beaad4f10f2fc5d7f9e43bc29170568c52850c2
Successfully built iapws
Коэффициент: 0.668521593152601
КПД: 40.145 %
Массовый расход в турбину на входе 275.8179044765468
Массовый расход в конденсатор: 115.53893749386874
